<a href="https://colab.research.google.com/github/mehdihosseinimoghadam/Signal-Processing/blob/main/WaveNet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Link for the Mind Map of WaveNet Structure 
https://miro.com/app/board/uXjVOP_ISaI=/?invite_link_id=623522214701

![picture](https://drive.google.com/uc?export=view&id=1732KrnQybOu00ldmJuf73aUyEiDnZ0jN)


In [2]:
import math
import pathlib
import random
import itertools
from tqdm import tqdm

from IPython import display
from dataclasses import dataclass

import torch
import torch.nn.functional as F
from torch import distributions
from torch import nn
from torch import optim
from torch.utils.data import Dataset, DataLoader

import torchaudio
from torchaudio.transforms import MelSpectrogram

import librosa
import pandas as pd
from sklearn.model_selection import train_test_split
from matplotlib import pyplot as plt

In [3]:
batch_size = 1
in_channel = 1
num_samples = 10

input = torch.arange(num_samples).reshape(batch_size, in_channel, num_samples).float()
print(f'input: {input}')

# Let's omit bias for simplicy
conv_1d = nn.Conv1d(in_channel, in_channel, 3, bias=False)
print(f'weights: {conv_1d.weight}')

# Turn weights into `ones`
nn.init.ones_(conv_1d.weight)

input: tensor([[[0., 1., 2., 3., 4., 5., 6., 7., 8., 9.]]])
weights: Parameter containing:
tensor([[[-0.0789,  0.0368, -0.1076]]], requires_grad=True)


Parameter containing:
tensor([[[1., 1., 1.]]], requires_grad=True)

In [ ]:
print(f'Input: {input.squeeze()}')
print(f'After Conv1D: {conv_1d(input).squeeze()}')

Input: tensor([0., 1., 2., 3., 4., 5., 6., 7., 8., 9.])
After Conv1D: tensor([ 3.,  6.,  9., 12., 15., 18., 21., 24.], grad_fn=<SqueezeBackward0>)


In [ ]:
class CausalConv1d(nn.Conv1d):
    """
    Casual Conv1d
    """

    def __init__(
        self,
        in_channels: int,
        out_channels: int,
        kernel_size: int,
        dilation: int = 1,
        bias: bool = False
    ):
        super().__init__(
            in_channels,
            out_channels,
            kernel_size,
            dilation=dilation,
            bias=bias
        )

        padding_size = (kernel_size - 1) * dilation
        self.zero_padding = nn.ConstantPad1d(
            padding=(padding_size, 0),
            value=0.0
        )

    def forward(self, input: torch.Tensor) -> torch.Tensor:
        # print(input)
        padded_input = self.zero_padding(input)
        # print(padded_input)
        output = super().forward(padded_input)
        # print(output)
        return output

In [ ]:
causal_block = CausalConv1d(in_channels=1, out_channels=1, kernel_size=3, dilation=3)
nn.init.ones_(causal_block.weight)

Parameter containing:
tensor([[[1., 1., 1.]]], requires_grad=True)

In [ ]:
print(f'Input: {input.squeeze()}')
print(f'After Conv1D: {causal_block(input).squeeze()}')

Input: tensor([0., 1., 2., 3., 4., 5., 6., 7., 8., 9.])
After Conv1D: tensor([ 0.,  1.,  2.,  3.,  5.,  7.,  9., 12., 15., 18.],
       grad_fn=<SqueezeBackward0>)


In [ ]:
class GatedConv1d(nn.Module):
  def __init__(
      self,
      in_channels,
      out_channels,
      kernel_size,
      dilation
  ):
      super().__init__()
      self.filter_conv = CausalConv1d(in_channels, out_channels, kernel_size, dilation)
      self.gated_conv = CausalConv1d(in_channels, out_channels, kernel_size, dilation)

  def forward(self, x):
    # print(1)
    filter_conv = self.filter_conv(x)
    gated_conv = self.gated_conv(x)
    # print(filter_conv)
    out = torch.tanh(filter_conv) * torch.sigmoid(gated_conv)
    # print(2)
    return out   

In [ ]:
gated_conv_block = GatedConv1d(in_channels=1, out_channels=1, kernel_size=3, dilation=3)
gated_conv_block

GatedConv1d(
  (filter_conv): CausalConv1d(
    1, 1, kernel_size=(3,), stride=(1,), dilation=(3,), bias=False
    (zero_padding): ConstantPad1d(padding=(6, 0), value=0.0)
  )
  (gated_conv): CausalConv1d(
    1, 1, kernel_size=(3,), stride=(1,), dilation=(3,), bias=False
    (zero_padding): ConstantPad1d(padding=(6, 0), value=0.0)
  )
)

In [ ]:
print(f'Input: {input.squeeze()}')
print(f'After Conv1D: {gated_conv_block(input)}')

Input: tensor([0., 1., 2., 3., 4., 5., 6., 7., 8., 9.])
After Conv1D: tensor([[[0.0000, 0.1022, 0.2008, 0.2894, 0.2801, 0.2687, 0.2556, 0.3919,
          0.5254, 0.6441]]], grad_fn=<MulBackward0>)


In [ ]:
dilation_cycles = 3

dilation_depth = 10

for i in range(dilation_cycles * dilation_depth):
  print(2 ** (i % dilation_depth))

In [8]:
from torch import nn

In [11]:
nn.ModuleList([
            CondWaveNetBlock(10, 20, 30,
                             40,
                             3, 2 ** (i % dilation_depth))
            for i in range(dilation_cycles * dilation_depth)])

NameError: ignored